# Notebook 4: Cifar10 Classification in Pytorch Lightning

In this notebook, we will train an image classifier for the cifar10 dataset, that you already know from exercise 6. Today, however, we will use the PyTorch Lightning framework which makes everything much more convenient!

In case you haven't done yet, you should definitely check out the **PyTorch Lightning Introduction** in **Exercise 7**!

## Imports

In [1]:
import os
import numpy as np

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader, random_split
import torchvision
import torchvision.transforms as transforms
from pytorch_lightning import Trainer
from pytorch_lightning.callbacks import EarlyStopping

%load_ext autoreload
%autoreload 2

### Set up PyTorch Lightning

In [2]:
!pip install pytorch-lightning==0.7.6 > /dev/null
import pytorch_lightning as pl

### Get Device
In this exercise, we'll use PyTorch Lightning to build an image classifier for the cifar10 dataset. As you know from exercise 06, processing a large set of images is quite computation extensive. Luckily, with PyTorch we're now able to make use of our GPU to significantly speed things up!

In case you don't have a GPU, you can run this notebook on Google Colab where you can access a GPU for free! 

Of course, you can also run this notebook on your CPU only - though this is definitely not recommended.


In [3]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

cpu


## Setup TensorBoard
In exercise 07 you've already learned how to use TensorBoard. Let's use it again to make the debugging of our network and training process more convenient! Throughout this notebook, feel free to add further logs or visualizations to your TensorBoard!

In [4]:
%load_ext tensorboard
%tensorboard --logdir lightning_logs

Reusing TensorBoard on port 6006 (pid 12250), started 21:27:22 ago. (Use '!kill 12250' to kill it.)

## Define your Network

Do you remember the good old times when we used to implement everything in plain numpy?

Luckliy, these times are over and we're using PyTorch Lightning which makes everything MUCH easier!

Instead of implementing your own model, solver and dataloader, all you have to do is defining a `LightningModule`.

We've prepared the class `exercise_code/MyPytorchModel` for you, that you'll now finalize to build an image classifier with PyTorch Lightning.

### 0. Dataset & Dataloaders
Check out the function `prepare_data` that loads the dataset, using the class `torchvision.datasets.ImageFolder`, which is very similar to the class `ImageFolderDataset` that you implemented earlier! 

Implement a **transform** to pre-process the raw data (standardize it and convert it to tensors) and assign it to the variable `my_transform`.

If you want, you can also perform **data augmentation** here!

Also check out the `DataLoader` class that is used to create  `train_dataloader` and `val_dataloader` and is very similar to your previous implementation of the DataLoader.

### 1. Define your model
Next, let's define your model. 

Think about a good network architecture. You're completely free here and can come up with any network you like! (\*)

Have a look at the documentation of `torch.nn` at https://pytorch.org/docs/stable/nn.html to learn how to use use this module to build your network!

Then implement your architecture: initialize it in `__init__()` and assign it to `self.model`. This is particularly easy using `nn.Sequential()` which you only have to pass the list of your layers.

Here's an easy example:

```python
        self.model = nn.Sequential(
            nn.Linear(input_size, self.hparams["n_hidden"]),
            nn.ReLU(),            
            nn.Linear(self.hparams["n_hidden"], num_classes)
        )
```

Have a look at the forward path in `forward(self, x)`, which is so easy, that you don't need to implement it yourself. 

As PyTorch automatically computes the gradients, that's all we need to do! No need anymore to manually calculate derivatives for the backward paths! :)

To make your model customizable and support parameter search, don't use hardcoded hyperparameters - instead, pass them as dictionary `hparams` when initializing `MyPytorchModel`.

____
\* *The size of your final model must be less than 20 MB, which is approximately equivalent to 5 Mio. params. Note that this limit is quite lenient, you will probably need much less parameters!*

*Also, don't use conv layers as they've not been covered yet in the lecture and build you network with fully connected layers (```nn.Linear()```)!*

### 2. Training & Validation Step
Have a look at the functions `training_step` and `validation_step`, that take a batch as input and calculate the loss. 

### 3. Optimizer
Lastly, implement the function `configure_optimizers()` to define your optimizer. Here the documentation of `torch.optim`at https://pytorch.org/docs/stable/optim.html might be helpful.


That's it! You've now finalized your `LightningModule` which has (at least) the same functionality as your previous numpy-powered image classifier!

Now let's create an instance of your `MyPytorchModel`.

In [52]:
from exercise_code.MyPytorchModel import MyPytorchModel

hparams = {}
########################################################################
# TODO: Define your hyper parameters here!                             #
########################################################################

hparams = {"n_hidden": 256, "batch_size": 64, "learning_rate": 0.00024206929900418654, "weight_decay": 1.6747171182248828e-07}

########################################################################
#                           END OF YOUR CODE                           #
########################################################################
model = MyPytorchModel(hparams)
model = model.to(device)
model.prepare_data()

Some tests to check whether we'll accept your model.

In [53]:
from exercise_code.Util import printModelInfo
_ = printModelInfo(model)

FYI: Your model has 0.855 params.
Model accepted!


## Fit Model with Trainer
Now it's time to train your model. 

Have a look of the documentation of `pl.Trainer` at https://pytorch-lightning.readthedocs.io/en/latest/trainer.html to find out what arguments you can pass to define your training process. 

Then, start the training with `trainer.fit(model)`.

In [54]:
trainer = None

########################################################################
# TODO: Define your trainer!                                           #
########################################################################
early_stop_callback = EarlyStopping(
    monitor='val_loss',
    patience=5,
    strict=False,
    verbose=False,
    mode='min'
)

trainer = Trainer(early_stop_callback=early_stop_callback)

########################################################################
#                           END OF YOUR CODE                           #
########################################################################

trainer.fit(model)

GPU available: False, used: False
INFO:lightning:GPU available: False, used: False
No environment variable for node rank defined. Set as 0.

  | Name    | Type       | Params
-----------------------------------
0 | model   | Sequential | 855 K 
1 | model.0 | Linear     | 786 K 
2 | model.1 | Dropout    | 0     
3 | model.2 | ReLU       | 0     
4 | model.3 | Linear     | 65 K  
5 | model.4 | Dropout    | 0     
6 | model.5 | ReLU       | 0     
7 | model.6 | Linear     | 2 K   
INFO:lightning:
  | Name    | Type       | Params
-----------------------------------
0 | model   | Sequential | 855 K 
1 | model.0 | Linear     | 786 K 
2 | model.1 | Dropout    | 0     
3 | model.2 | ReLU       | 0     
4 | model.3 | Linear     | 65 K  
5 | model.4 | Dropout    | 0     
6 | model.5 | ReLU       | 0     
7 | model.6 | Linear     | 2 K   


Val-Acc=0.0014


Val-Acc=0.4345


Val-Acc=0.4644


Val-Acc=0.482


...

Val-Acc=0.5116


Val-Acc=0.5209


Val-Acc=0.5252


Val-Acc=0.5211



1

Now that everything is working, feel free to play around with different architectures. As you've seen, it's really easy to define your model or do changes there.

You can now checkout notebook 5 which shows you how you can use the framework Optuna to perform **hyperparameter tuning with PyTorch lightning!**

To pass this submission, you'll need **50%** accuracy.

# Save your model & Report Test Accuracy

When you've done with your **hyperparameter tuning (see notebook 5)**, have achieved **at least 50% validation accuracy** and are happy with your final model, you can save it here.

Before that, we will check again whether the number of parameters is below 5 Mio and the file size is below 20 MB.

When your final model is saved, we'll lastly report the test accuracy.

In [55]:
from exercise_code.Util import test_and_save
test_and_save(model)

Validation-Accuracy: 0.5015%
FYI: Your model has 0.855 params.
Great! Your model size is less than 20 MB and will be accepted :)
Your model has been saved and is ready to be submitted. NOW, let's check the test-accuracy.
Test-Accuracy: 0.5104%


Congrats! You've now finished your first image classifier in PyTorch Lightning! Much easier, than in plain numpy, right? Time to get started with some more complex neural networks - see you at the next exercise!

# Submission Instructions

Congratulations! You've just built your first image classifier with PyTorch Lightning! To complete the exercise, submit your final model to our submission portal - you probably know the procedure by now.

1. Go on [our submission page](https://dvl.in.tum.de/teaching/submission/), register for an account and login. We use your matriculation number and send an email with the login details to the mail account associated. When in doubt, login into tum online and check your mails there. You will get an ID which we need in the next step.
2. Navigate to `exercise_code` directory and run the `create_submission.sh` file to create the zip file of your model. This will create a single `zip` file that you need to upload. Otherwise, you can also zip it manually if you don't want to use the bash script. However, **make sure that the structure of the zip file is the same** as it would be when generated with the bash-script.
3. Log into [our submission page](https://dvl.in.tum.de/teaching/submission/) with your account details and upload the `zip` file. Once successfully uploaded, you should be able to see the submitted file selectable on the top.
4. Click on this file and run the submission script. You will get an email with your score as well as a message if you have surpassed the threshold.

# Submission Goals

- Goal: Successfully implement a a fully connected NN image classifier for cifar10 with PyTorch Lightning

- Passing Criteria: This time, there are no unit tests that check specific components of your code. The only thing that's required to pass the submission, is your model to reach at least **50% accuracy** on __our__ test dataset. The submission system will show you a number between 0 and 100 which corresponds to your accuracy.

- Submission start: __Thursday, June 11, 2020 12.00__
- Submission deadline : __June 17, 2020 23.59__ 
- You can make **$\infty$** submissions until the deadline. Your __best submission__ will be considered for bonus